# Add predicted domains DUFLT

Version février 2026 intégrant les 40 domaines de Denis

In [31]:
from transformers import BertForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch
from tqdm.auto import tqdm
import os
import datetime

# date as "yyyymmdd"
suffix = datetime.datetime.now().strftime("%y%m%d")
suffix

'260211'

In [32]:

# 1. Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

TEXT_COLUMN = "text"

# 2. Load Model and Tokenizer
MODEL_PATH = "../models/final_multiclass_model"
model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model.to(device)
model.eval() # Set to evaluation mode

# 3. Create a simple Dataset class
class SimpleDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

def predict_dataframe(df, text_column, batch_size=16):
    texts = df[text_column].astype(str).tolist()
    dataset = SimpleDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    all_preds = []
    
    # Wrap dataloader with tqdm for the progress bar
    # 'total' helps tqdm calculate the percentage correctly
    progress_bar = tqdm(dataloader, desc="Predicting", unit="batch")
    
    with torch.no_grad():
        for batch in progress_bar:
            inputs = tokenizer(
                batch, 
                padding=True, 
                truncation=True, 
                max_length=512, 
                return_tensors="pt"
            ).to(device)
            
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, axis=-1).cpu().numpy()
            all_preds.extend(preds)
            
            # Optional: Add real-time info to the bar
            # progress_bar.set_postfix({"last_batch_size": len(preds)})

    df['predicted_label_id'] = all_preds
    df['predDomain'] = df['predicted_label_id'].apply(lambda x: model.config.id2label[x])
    return df



In [29]:
sentence = "Brin , se dit aussi des menus jets des herbes, des joncs, des cheveux, & de tout ce que des racines poussent. Coliculus, surculus. Il faut mettre deux ou trois brins de ciboulette dans cette salade. Il n'est resté à ce convalescent que deux ou trois brins de cheveux. Les brins des vergettes sont faits de petits joncs. Les tresses de cheveux se font brin à brin. Philyra."

inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
outputs = model(**inputs)
pred = torch.argmax(outputs.logits, axis=-1).cpu().item()
pred_label = model.config.id2label[pred]
print(f"Predicted label: {pred_label} (ID: {pred})")

Predicted label: Métiers [alimentation] (ID: 26)


In [33]:
df = pd.read_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743_paragraphs_macro_260211.tsv'), sep='\t', encoding='utf-8')
df.head()

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain
0,DUFLT_1743,1,1,A,1,mainEntry,1,NaN,A est la première Lettre de l'Alphabet Françoi...,0,NaN
1,DUFLT_1743,1,1,A,1,mainEntry,2,NaN,C'est inutilement que la plupart des Grammairi...,0,NaN
2,DUFLT_1743,1,1,A,1,mainEntry,3,NaN,"A se prononce du gozier, ce qui ne rend pas ce...",0,NaN
3,DUFLT_1743,1,1,A,1,mainEntry,4,NaN,Le son de l'a est ordinairement un son clair. ...,0,NaN
4,DUFLT_1743,1,1,A,1,mainEntry,5,NaN,Le son de l'a est un de ceux que les muets for...,0,NaN


In [34]:
df.shape

(112053, 11)

In [35]:
# get a sample of rows where text is longer than 100 characters or src-domain is not null
df_sample = df[(df['text'].str.len() > 100) | (df['srcDomain'].isna() == False)]
df_sample

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain
0,DUFLT_1743,1,1,A,1,mainEntry,1,NaN,A est la première Lettre de l'Alphabet Françoi...,0,NaN
1,DUFLT_1743,1,1,A,1,mainEntry,2,NaN,C'est inutilement que la plupart des Grammairi...,0,NaN
2,DUFLT_1743,1,1,A,1,mainEntry,3,NaN,"A se prononce du gozier, ce qui ne rend pas ce...",0,NaN
3,DUFLT_1743,1,1,A,1,mainEntry,4,NaN,Le son de l'a est ordinairement un son clair. ...,0,NaN
4,DUFLT_1743,1,1,A,1,mainEntry,5,NaN,Le son de l'a est un de ceux que les muets for...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
112046,DUFLT_1743,6,5581,ZYGETH,2,relatedEntry,1,NaN,"ZYGETH Zygeth , est aussi un village de la Hau...",0,NaN
112047,DUFLT_1743,6,5582,ZYGÔME,1,mainEntry,1,medecine & d'anatomie,ZYGÔME Terme de Médecine & d'Anatomie . Orzygo...,1,Médecine
112048,DUFLT_1743,6,5583,ZYMOSIMÈTRE,1,mainEntry,1,NaN,ZYMOSIMÈTRE est un instrument proposé par Swam...,0,NaN
112050,DUFLT_1743,6,5584,ZYP,1,mainEntry,1,NaN,ZYP C'étoit autrefois un grand marais de la No...,0,NaN


In [36]:
df_sample_pred = predict_dataframe(df_sample, TEXT_COLUMN, batch_size=16)

Predicting:   0%|          | 0/5903 [00:00<?, ?batch/s]

/var/folders/qm/v_b1md29221_cnpcxf5qc43c0000gn/T/ipykernel_1397/317364521.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_label_id'] = all_preds
/var/folders/qm/v_b1md29221_cnpcxf5qc43c0000gn/T/ipykernel_1397/317364521.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predDomain'] = df['predicted_label_id'].apply(lambda x: model.config.id2label[x])


In [37]:
df_sample.head()

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain,predicted_label_id,predDomain
0,DUFLT_1743,1,1,A,1,mainEntry,1,NaN,A est la première Lettre de l'Alphabet Françoi...,0,NaN,6,Belles-lettres
1,DUFLT_1743,1,1,A,1,mainEntry,2,NaN,C'est inutilement que la plupart des Grammairi...,0,NaN,12,Grammaire
2,DUFLT_1743,1,1,A,1,mainEntry,3,NaN,"A se prononce du gozier, ce qui ne rend pas ce...",0,NaN,12,Grammaire
3,DUFLT_1743,1,1,A,1,mainEntry,4,NaN,Le son de l'a est ordinairement un son clair. ...,0,NaN,12,Grammaire
4,DUFLT_1743,1,1,A,1,mainEntry,5,NaN,Le son de l'a est un de ceux que les muets for...,0,NaN,25,Médecine


In [38]:
df.loc[df_sample_pred.index, 'predDomain'] = df_sample_pred['predDomain']

In [39]:
df

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain,predDomain
0,DUFLT_1743,1,1,A,1,mainEntry,1,NaN,A est la première Lettre de l'Alphabet Françoi...,0,NaN,Belles-lettres
1,DUFLT_1743,1,1,A,1,mainEntry,2,NaN,C'est inutilement que la plupart des Grammairi...,0,NaN,Grammaire
2,DUFLT_1743,1,1,A,1,mainEntry,3,NaN,"A se prononce du gozier, ce qui ne rend pas ce...",0,NaN,Grammaire
3,DUFLT_1743,1,1,A,1,mainEntry,4,NaN,Le son de l'a est ordinairement un son clair. ...,0,NaN,Grammaire
4,DUFLT_1743,1,1,A,1,mainEntry,5,NaN,Le son de l'a est un de ceux que les muets for...,0,NaN,Médecine
...,...,...,...,...,...,...,...,...,...,...,...,...
112048,DUFLT_1743,6,5583,ZYMOSIMÈTRE,1,mainEntry,1,NaN,ZYMOSIMÈTRE est un instrument proposé par Swam...,0,NaN,Sciences physico-mathématiques
112049,DUFLT_1743,6,5583,ZYMOSIMÈTRE,1,mainEntry,2,NaN,"Ce mot vient de , fermentatio, fermentation, & .",0,NaN,NaN
112050,DUFLT_1743,6,5584,ZYP,1,mainEntry,1,NaN,ZYP C'étoit autrefois un grand marais de la No...,0,NaN,Géographie
112051,DUFLT_1743,6,5585,ZYTHI,1,mainEntry,1,NaN,ZYTHI Voyez Zuthi.,0,NaN,NaN


In [40]:
path = os.path.join('..', 'data', '1743_LeRobert')
df.to_csv(os.path.join(path, "Trevoux1743_paragraphs_macro_pred_"+suffix+".tsv"), sep="\t", index=False)
df.to_excel(os.path.join(path, "Trevoux1743_paragraphs_macro_pred_"+suffix+".xlsx"), index=False)